In [ ]:

"""
SCRIPT COMPLETO: Extracción de Datos Agrícolas para Videojuego
Región: Cusco, Perú
Datos: NDVI, Temperatura, Precipitación
Autor: Para Hackathon FarmGuardians
"""

# ============================================
# PASO 1: INSTALACIÓN Y CONFIGURACIÓN
# ============================================
# Ejecutar en Colab:
# !pip install earthengine-api geemap pandas numpy matplotlib

import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ============================================
# PASO 2: AUTENTICACIÓN
# ============================================
# Primera vez: ee.Authenticate()
ee.Initialize(project='tu-proyecto-aqui')  # Cambiar por tu proyecto

# ============================================
# PASO 3: DEFINIR ÁREA DE CUSCO
# ============================================

# Opción A: Punto específico (Plaza de Armas, Cusco)
cusco_point = ee.Geometry.Point([-71.9675, -13.5319])

# Opción B: Región completa de Cusco
cusco_region = ee.Geometry.Polygon([
    [-72.5, -13.0],
    [-71.0, -13.0],
    [-71.0, -14.5],
    [-72.5, -14.5],
    [-72.5, -13.0]
])

# Opción C: Valle Sagrado (zona agrícola importante)
valle_sagrado = ee.Geometry.Polygon([
    [-72.2, -13.2],
    [-71.8, -13.2],
    [-71.8, -13.4],
    [-72.2, -13.4],
    [-72.2, -13.2]
])

# Elegir geometría para análisis
area_estudio = cusco_point  # Cambiar según necesites

# ============================================
# PASO 4: FUNCIONES DE EXTRACCIÓN
# ============================================

def extraer_ndvi_sentinel2(geometry, fecha_inicio, fecha_fin):
    """
    Extrae NDVI de Sentinel-2 (10m resolución)
    Ideal para parcelas específicas
    """
    def calcular_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return ndvi.copyProperties(image, ['system:time_start'])

    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate(fecha_inicio, fecha_fin) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
        .map(calcular_ndvi)

    return collection


def extraer_ndvi_modis(geometry, fecha_inicio, fecha_fin):
    """
    Extrae NDVI de MODIS (250m resolución)
    Mejor cobertura temporal, datos desde 2000
    """
    collection = ee.ImageCollection('MODIS/061/MOD13Q1') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('NDVI')

    def escalar_ndvi(image):
        return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

    return collection.map(escalar_ndvi)


def extraer_temperatura(geometry, fecha_inicio, fecha_fin):
    """
    Extrae temperatura superficial (°C)
    """
    collection = ee.ImageCollection('MODIS/061/MOD11A2') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('LST_Day_1km')

    def convertir_celsius(image):
        celsius = image.multiply(0.02).subtract(273.15)
        return celsius.copyProperties(image, ['system:time_start'])

    return collection.map(convertir_celsius)


def extraer_precipitacion(geometry, fecha_inicio, fecha_fin):
    """
    Extrae precipitación diaria (mm)
    """
    collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('precipitation')

    return collection


def collection_a_dataframe(image_collection, geometry, escala, nombre_banda):
    """
    Convierte ImageCollection a DataFrame de pandas
    """
    def extraer_valor(image):
        valor = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=escala,
            maxPixels=1e9
        ).get(nombre_banda)

        fecha = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

        return ee.Feature(None, {
            'fecha': fecha,
            nombre_banda: valor
        })

    features = image_collection.map(extraer_valor)
    info = features.getInfo()

    datos = [f['properties'] for f in info['features']]
    df = pd.DataFrame(datos)
    df['fecha'] = pd.to_datetime(df['fecha'])

    return df


# ============================================
# PASO 5: EXTRAER DATOS (2023-2024)
# ============================================

print("🌱 Extrayendo datos de Cusco para videojuego...")

fecha_inicio = '2023-01-01'
fecha_fin = '2024-12-31'

# NDVI (usa Sentinel-2 para alta resolución)
print("📊 Extrayendo NDVI...")
ndvi_collection = extraer_ndvi_sentinel2(area_estudio, fecha_inicio, fecha_fin)
ndvi_df = collection_a_dataframe(ndvi_collection, area_estudio, 10, 'NDVI')

# Temperatura
print("🌡️ Extrayendo temperatura...")
temp_collection = extraer_temperatura(area_estudio, fecha_inicio, fecha_fin)
temp_df = collection_a_dataframe(temp_collection, area_estudio, 1000, 'LST_Day_1km')
temp_df.rename(columns={'LST_Day_1km': 'temperatura_celsius'}, inplace=True)

# Precipitación
print("💧 Extrayendo precipitación...")
precip_collection = extraer_precipitacion(area_estudio, fecha_inicio, fecha_fin)
precip_df = collection_a_dataframe(precip_collection, area_estudio, 5000, 'precipitation')
precip_df.rename(columns={'precipitation': 'precipitacion_mm'}, inplace=True)

print("✅ Datos extraídos exitosamente!")

# ============================================
# PASO 6: COMBINAR Y LIMPIAR DATOS
# ============================================

# Unir todos los dataframes
datos_completos = ndvi_df.merge(temp_df, on='fecha', how='outer') \
                         .merge(precip_df, on='fecha', how='outer')

# Ordenar por fecha
datos_completos = datos_completos.sort_values('fecha').reset_index(drop=True)

# Interpolar valores faltantes
datos_completos['NDVI'] = datos_completos['NDVI'].interpolate(method='linear')
datos_completos['temperatura_celsius'] = datos_completos['temperatura_celsius'].interpolate(method='linear')
datos_completos['precipitacion_mm'] = datos_completos['precipitacion_mm'].fillna(0)

# ============================================
# PASO 7: CREAR MÉTRICAS PARA EL JUEGO
# ============================================

def calcular_metricas_juego(df):
    """
    Crea métricas útiles para mecánicas de videojuego
    """
    # Salud del cultivo (0-100)
    df['salud_cultivo'] = (df['NDVI'] * 100).clip(0, 100)

    # Estrés por calor
    df['estres_calor'] = pd.cut(
        df['temperatura_celsius'],
        bins=[-np.inf, 15, 25, 35, np.inf],
        labels=['Frío', 'Óptimo', 'Caluroso', 'Extremo']
    )

    # Necesidad de riego
    df['necesidad_riego'] = pd.cut(
        df['precipitacion_mm'],
        bins=[-np.inf, 5, 15, 30, np.inf],
        labels=['Crítico', 'Alto', 'Moderado', 'Bajo']
    )

    # Índice de productividad (combinado)
    df['productividad'] = (
        df['salud_cultivo'] * 0.5 +
        (100 - abs(df['temperatura_celsius'] - 20) * 2).clip(0, 100) * 0.3 +
        df['precipitacion_mm'].clip(0, 50) * 0.4
    ).clip(0, 100)

    # Estación del año (hemisferio sur)
    df['mes'] = df['fecha'].dt.month
    estaciones = {
        12: 'Verano', 1: 'Verano', 2: 'Verano',
        3: 'Otoño', 4: 'Otoño', 5: 'Otoño',
        6: 'Invierno', 7: 'Invierno', 8: 'Invierno',
        9: 'Primavera', 10: 'Primavera', 11: 'Primavera'
    }
    df['estacion'] = df['mes'].map(estaciones)

    return df

datos_juego = calcular_metricas_juego(datos_completos)

# ============================================
# PASO 8: EXPORTAR DATOS
# ============================================

# CSV para análisis
datos_juego.to_csv('cusco_datos_agricolas_juego.csv', index=False)
print("💾 Datos guardados en: cusco_datos_agricolas_juego.csv")

# JSON para Unity/Godot/Unreal
datos_json = datos_juego.to_dict(orient='records')
import json
with open('cusco_datos_agricolas_juego.json', 'w', encoding='utf-8') as f:
    json.dump(datos_json, f, indent=2, default=str, ensure_ascii=False)
print("💾 Datos guardados en: cusco_datos_agricolas_juego.json")

# ============================================
# PASO 9: VISUALIZACIÓN
# ============================================

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# NDVI
axes[0].plot(datos_juego['fecha'], datos_juego['NDVI'], color='green', linewidth=2)
axes[0].set_title('NDVI - Salud de la Vegetación en Cusco', fontsize=14, fontweight='bold')
axes[0].set_ylabel('NDVI')
axes[0].grid(True, alpha=0.3)

# Temperatura
axes[1].plot(datos_juego['fecha'], datos_juego['temperatura_celsius'], color='red', linewidth=2)
axes[1].set_title('Temperatura Superficial (°C)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('°C')
axes[1].grid(True, alpha=0.3)

# Precipitación
axes[2].bar(datos_juego['fecha'], datos_juego['precipitacion_mm'], color='blue', alpha=0.6)
axes[2].set_title('Precipitación Diaria (mm)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('mm')
axes[2].grid(True, alpha=0.3)

# Productividad (métrica del juego)
axes[3].plot(datos_juego['fecha'], datos_juego['productividad'], color='purple', linewidth=2)
axes[3].set_title('Índice de Productividad (Métrica del Juego)', fontsize=14, fontweight='bold')
axes[3].set_ylabel('Índice (0-100)')
axes[3].set_xlabel('Fecha')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('cusco_analisis_agricola.png', dpi=300, bbox_inches='tight')
print("📊 Gráfico guardado en: cusco_analisis_agricola.png")
plt.show()

# ============================================
# PASO 10: ESTADÍSTICAS PARA DOCUMENTACIÓN
# ============================================

print("\n" + "="*60)
print("📈 ESTADÍSTICAS PARA TU HACKATHON")
print("="*60)

print(f"\n📅 Período analizado: {datos_juego['fecha'].min()} a {datos_juego['fecha'].max()}")
print(f"📊 Total de observaciones: {len(datos_juego)}")

print("\n🌱 NDVI (Salud de Cultivos):")
print(f"   • Promedio: {datos_juego['NDVI'].mean():.3f}")
print(f"   • Máximo: {datos_juego['NDVI'].max():.3f}")
print(f"   • Mínimo: {datos_juego['NDVI'].min():.3f}")

print("\n🌡️ Temperatura:")
print(f"   • Promedio: {datos_juego['temperatura_celsius'].mean():.1f}°C")
print(f"   • Máxima: {datos_juego['temperatura_celsius'].max():.1f}°C")
print(f"   • Mínima: {datos_juego['temperatura_celsius'].min():.1f}°C")

print("\n💧 Precipitación:")
print(f"   • Total anual: {datos_juego['precipitacion_mm'].sum():.1f} mm")
print(f"   • Promedio diario: {datos_juego['precipitacion_mm'].mean():.1f} mm")
print(f"   • Día más lluvioso: {datos_juego['precipitacion_mm'].max():.1f} mm")

print("\n🎮 Métricas del Juego:")
print(f"   • Productividad promedio: {datos_juego['productividad'].mean():.1f}/100")
print(f"   • Días con estrés extremo: {(datos_juego['estres_calor'] == 'Extremo').sum()}")
print(f"   • Días con riego crítico: {(datos_juego['necesidad_riego'] == 'Crítico').sum()}")

print("\n✅ ¡Datos listos para tu videojuego!")
print("="*60)

# 🌍 Datos Satelitales Disponibles en Google Earth Engine (GEE)

En este proyecto usaremos diferentes variables satelitales que afectan la agricultura y pueden ser integradas al videojuego.

| Indicador / Variable            | Dataset en GEE                                   | Resolución | Qué mide (unidad) | Uso en Agricultura / Juego |
|---------------------------------|--------------------------------------------------|------------|-------------------|-----------------------------|
| **NDVI** (vegetación)           | `COPERNICUS/S2_SR_HARMONIZED` (Sentinel-2)       | 10 m       | Índice de vegetación (adimensional, -1 a 1) | Salud del cultivo, estrés |
| **EVI** (vegetación mejorada)   | `MODIS/061/MOD13Q1`                              | 250 m      | Índice de vegetación ajustado (adimensional, -1 a 1) | Más robusto en zonas densas |
| **Anomalías NDVI históricas**   | MODIS/Sentinel (comparando con medias históricas) | 250 m–10 m | Diferencia NDVI actual - promedio histórico | Detectar bajas campañas |
| **Temperatura superficial**     | `MODIS/061/MOD11A2`                              | 1 km       | Temperatura de la superficie (°C) | Estrés térmico, olas de calor |
| **Temperatura mínima / máxima** | `ECMWF/ERA5_LAND/HOURLY`                         | 9 km       | Temperatura del aire 2m (°C) | Heladas y calor extremo |
| **Precipitación diaria**        | `UCSB-CHG/CHIRPS/DAILY`                          | 5 km       | Lluvia acumulada diaria (mm/día) | Estimar agua disponible |
| **Anomalías de precipitación**  | CHIRPS histórico (comparando series temporales)  | 5 km       | Diferencia lluvia actual - promedio histórico (mm) | Fenómeno El Niño / La Niña |
| **Humedad del suelo**           | `NASA_USDA/HSL/SMAP10KM_soil_moisture`           | 10 km      | Humedad volumétrica del suelo (m³/m³) | Disponibilidad real de agua |
| **Evapotranspiración (ET)**     | `MODIS/061/MOD16A2`                              | 500 m      | Agua transferida suelo–atmósfera (mm/8 días) | Demanda hídrica del cultivo |
| **Índice de sequía (NDWI, ESI)**| MODIS/Sentinel                                   | 10–500 m   | Balance agua en vegetación (adimensional) | Estrés hídrico en plantas |
| **Biomasa (LAI, FAPAR)**        | `MODIS/061/MCD15A3H` (LAI/FAPAR)                 | 500 m      | LAI = área foliar (m²/m²), FAPAR = fracción de radiación absorbida (%) | Productividad fotosintética |
| **Cobertura terrestre**         | `MODIS/061/MCD12Q1`                              | 500 m      | Clasificación de uso del suelo (categorías) | Identificar uso agrícola del suelo |
| **Incendios / focos de calor**  | `FIRMS` (`FIRMS`)                                | 375 m      | Radiación térmica de incendios (MW) | Impacto en parcelas agrícolas |
| **Nubes persistentes**          | `COPERNICUS/S2_CLOUD_PROBABILITY`                | 10 m       | Probabilidad de nubosidad (%) | Interferencia en fotosíntesis |
| **Aerosoles / Contaminación**   | `COPERNICUS/S5P/OFFL/L3_AER_AI`                  | 1 km       | Profundidad óptica de aerosoles (adimensional) | Estrés ambiental en cultivos |

---

📌 **Nota sobre El Niño (ENOS):**  
GEE no incluye directamente el índice ONI (temperatura del Pacífico central), pero sí podemos usar **anomalías de precipitación (CHIRPS)** y **temperaturas ERA5** como indicadores indirectos.


# 🌾 Datos Satelitales para FarmGuardians

## Datos que se pueden extraer directamente (crudos)

- 🌱 **NDVI y otras bandas de vegetación** (Sentinel-2, MODIS)  
- 🌡️ **Temperatura superficial diurna y nocturna** (MODIS: LST)  
- 💧 **Precipitación diaria** (CHIRPS, GPM)  
- 🌍 **Cobertura del suelo, humedad superficial del suelo** (ERA5-Land, SMAP)  
- ❄️ **Temperatura mínima / días de helada** (de LST)  
- 🌬️ **Evapotranspiración, radiación solar** (ERA5-Land, MODIS)  

## Datos que solo se pueden calcular / derivar

- Estrés hídrico y necesidad de riego  
- Estrés térmico y días de calor o frío extremo  
- Índices de productividad agrícola  
- Alertas de riesgo de plagas usando proxies (NDVI, temperatura, humedad)  
- Detección de anomalías de vegetación e inundaciones


In [ ]:
# Clonar el repositorio desde GitHub
!git clone https://github.com/iAmMazapan/FarmGuardians.git

# Instalar las dependencias desde requirements.txt
!pip install -r FarmGuardians/DataIntegrator/requirements.txt


fatal: destination path 'FarmGuardians' already exists and is not an empty directory.


---

## Paso 2: Configurar accesos

En este paso vamos a:
1. **Montar Google Drive** - Para guardar los resultados
2. **Autenticar Google Earth Engine** - Para acceder a datos satelitales

### ¿Por qué necesitamos esto?

- **Google Drive**: Los datos extraídos (CSV, JSON) se guardarán aquí para no perderlos
- **Google Earth Engine**: Es la plataforma de Google que nos da acceso gratuito a petabytes de imágenes satelitales

### Notas importantes:
- Te pedirá permisos para acceder a tu Drive
- La autenticación de GEE es mediante una cuenta de Google
- Solo necesitas hacerlo una vez por sesión

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive/')

print("✅ Google Drive montado en /content/drive/")
print("-" * 50)

# 2. Autenticar Google Earth Engine
import ee

print("\n Iniciando autenticación de Google Earth Engine...")
print("Se abrirá una ventana para autenticarte con tu cuenta de Google")

# Autenticar (se abre ventana emergente)
ee.Authenticate()

# Inicializar (sin proyecto específico por ahora)
ee.Initialize(project='farmguardians-gee')

print(" Google Earth Engine autenticado correctamente")
print("-" * 50)
print("\n🎉 Todo listo para extraer datos satelitales!")

Mounted at /content/drive/
✅ Google Drive montado en /content/drive/
--------------------------------------------------

 Iniciando autenticación de Google Earth Engine...
Se abrirá una ventana para autenticarte con tu cuenta de Google
 Google Earth Engine autenticado correctamente
--------------------------------------------------

🎉 Todo listo para extraer datos satelitales!


---

## Paso 3: Definir área de estudio en Cusco

Vamos a definir las coordenadas de nuestra zona de interés en Cusco, Perú.

### Opciones disponibles:

1. **Punto específico** - Para analizar una parcela o campo concreto
2. **Región amplia** - Para analizar toda la zona de Cusco
3. **Valle Sagrado** - Zona agrícola importante

### ¿Qué vamos a usar?

Para este análisis inicial usaremos **un punto específico cerca de la ciudad de Cusco** que tiene actividad agrícola visible desde satélite.

**Coordenadas seleccionadas:**
- Latitud: -13.5319°
- Longitud: -71.9675°
- Ubicación: Zona agrícola cerca de Cusco

Puedes cambiar estas coordenadas después si quieres analizar otra zona específica.

In [ ]:
import ee
import geemap
from ipywidgets import HBox

# Lista de ubicaciones y rangos de fechas para extracción de datos

ubicaciones_fenomenos = [
    {
        "fenomeno": "Sequía extrema",
        "pais_region": "California, EE. UU.",
        "ubicacion": "Valle Central / Fresno",
        "coordenadas": [36.7378, -119.7871],  # coordenada aproximada central
        "fecha_inicio": "2014-01-01",
        "fecha_fin": "2017-12-31",
        "descripcion": "Sequía prolongada afectando agricultura e incendios forestales"
    },
    {
        "fenomeno": "Calor extremo",
        "pais_region": "Australia (Sur y Este)",
        "ubicacion": "Melbourne / Victoria",
        "coordenadas": [-37.8136, 144.9631],
        "fecha_inicio": "2019-01-01",
        "fecha_fin": "2020-12-31",
        "descripcion": "Ola de calor con temperaturas récord y estrés en cultivos"
    },
    {
        "fenomeno": "Plagas agrícolas",
        "pais_region": "India (Punjab)",
        "ubicacion": "Ludhiana",
        "coordenadas": [30.9000, 75.8573],
        "fecha_inicio": "2020-01-01",
        "fecha_fin": "2020-12-31",
        "descripcion": "Brotes de langostas del desierto afectando trigo y otros cultivos"
    },
    {
        "fenomeno": "Precipitaciones fluviales",
        "pais_region": "Bangladesh (Delta del Ganges)",
        "ubicacion": "Dhaka",
        "coordenadas": [23.6850, 90.3563],
        "fecha_inicio": "2017-01-01",
        "fecha_fin": "2017-12-31",
        "descripcion": "Inundaciones por lluvias monzónicas, afectando arrozales"
    },
    {
        "fenomeno": "Heladas / Frío extremo",
        "pais_region": "Rusia (Siberia)",
        "ubicacion": "Novosibirsk",
        "coordenadas": [55.0084, 82.9357],
        "fecha_inicio": "2018-01-01",
        "fecha_fin": "2018-12-31",
        "descripcion": "Heladas severas que afectaron cultivos de cereales y hortalizas"
    },
    {
        "fenomeno": "Fenómeno de El Niño / Lluvias intensas",
        "pais_region": "Perú (Costa norte y central)",
        "ubicacion": "Piura",
        "coordenadas": [-5.1945, -80.6328],
        "fecha_inicio": "2017-01-01",
        "fecha_fin": "2017-12-31",
        "descripcion": "Lluvias extremas, inundaciones y desbordes de ríos, afectando cultivos y suelos"
    },
    {
        "fenomeno": "Fenómeno de El Niño / Lluvias intensas",
        "pais_region": "Perú (Costa norte y central)",
        "ubicacion": "Lima",
        "coordenadas": [-12.0464, -77.0428],
        "fecha_inicio": "2017-01-01",
        "fecha_fin": "2017-12-31",
        "descripcion": "Lluvias extremas, inundaciones y desbordes de ríos, afectando cultivos y suelos"
    }
]



In [ ]:
# Instalar folium si no está instalado
!pip install folium

import folium

# Crear mapa centrado en coordenadas globales aproximadas
mundo_map = folium.Map(location=[0, 0], zoom_start=2)

# Lista de ubicaciones (la misma que definimos antes)
ubicaciones_fenomenos = [
    {"fenomeno": "Sequía extrema", "pais_region": "California, EE. UU.", "ubicacion": "Valle Central / Fresno", "coordenadas": [36.7378, -119.7871]},
    {"fenomeno": "Calor extremo", "pais_region": "Australia (Sur y Este)", "ubicacion": "Melbourne / Victoria", "coordenadas": [-37.8136, 144.9631]},
    {"fenomeno": "Plagas agrícolas", "pais_region": "India (Punjab)", "ubicacion": "Ludhiana", "coordenadas": [30.9000, 75.8573]},
    {"fenomeno": "Precipitaciones fluviales", "pais_region": "Bangladesh (Delta del Ganges)", "ubicacion": "Dhaka", "coordenadas": [23.6850, 90.3563]},
    {"fenomeno": "Heladas / Frío extremo", "pais_region": "Rusia (Siberia)", "ubicacion": "Novosibirsk", "coordenadas": [55.0084, 82.9357]},
    {"fenomeno": "Fenómeno de El Niño / Lluvias intensas", "pais_region": "Perú (Costa norte y central)", "ubicacion": "Piura", "coordenadas": [-5.1945, -80.6328]},
    {"fenomeno": "Fenómeno de El Niño / Lluvias intensas", "pais_region": "Perú (Costa norte y central)", "ubicacion": "Lima", "coordenadas": [-12.0464, -77.0428]}
]

# Añadir marcadores al mapa
for lugar in ubicaciones_fenomenos:
    folium.Marker(
        location=lugar["coordenadas"],
        popup=f"<b>País/Región:</b> {lugar['pais_region']}<br>"
              f"<b>Ubicación:</b> {lugar['ubicacion']}<br>"
              f"<b>Fenómeno:</b> {lugar['fenomeno']}",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(mundo_map)

# Mostrar mapa
mundo_map


---

## Paso 4: Definir funciones de extracción de datos


In [50]:
from datetime import datetime, timedelta

def ajustar_fechas(fecha_inicio, fecha_fin):
    fmt = "%Y-%m-%d"
    start = datetime.strptime(fecha_inicio, fmt)
    end = datetime.strptime(fecha_fin, fmt)

    diff_years = (end - start).days / 365.25

    if diff_years < 3:
        # Expandir inicio y fin para cubrir 3 años
        expand = (3 - diff_years)/2
        start = start - timedelta(days=int(expand*365))
        end = end + timedelta(days=int(expand*365))

    return start.strftime(fmt), end.strftime(fmt)

# Ajustar fechas para todas las localidades
for lugar in ubicaciones_fenomenos:
    lugar["fecha_inicio_ajustada"], lugar["fecha_fin_ajustada"] = ajustar_fechas(
        lugar["fecha_inicio"], lugar["fecha_fin"]
    )

# Verificar resultados
for lugar in ubicaciones_fenomenos:
    print(f"{lugar['ubicacion']}: {lugar['fecha_inicio_ajustada']} → {lugar['fecha_fin_ajustada']}")


Valle Central / Fresno: 2014-01-01 → 2017-12-31
Melbourne / Victoria: 2018-07-03 → 2021-07-01
Ludhiana: 2019-01-01 → 2021-12-31
Dhaka: 2016-01-02 → 2018-12-31
Novosibirsk: 2017-01-01 → 2019-12-31
Piura: 2016-01-02 → 2018-12-31
Lima: 2016-01-02 → 2018-12-31


---

## Paso 5: Extraer datos satelitales



In [53]:
def extraer_datos_ubicacion(lugar):
    geom = ee.Geometry.Point(lugar["coordenadas"])
    start = lugar["fecha_inicio_ajustada"]
    end = lugar["fecha_fin_ajustada"]

    # Buffer pequeño para asegurar cobertura
    region = geom.buffer(1000)  # 1km de buffer

    # Colecciones que vamos a usar
    ndvi_coll = ee.ImageCollection('MODIS/006/MOD13Q1').filterDate(start, end).filterBounds(geom).select('NDVI')
    lst_coll = ee.ImageCollection('MODIS/006/MOD11A1').filterDate(start, end).filterBounds(geom).select(['LST_Day_1km', 'LST_Night_1km'])
    precip_coll = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(start, end).filterBounds(geom).select('precipitation')

    # ERA5-Land para humedad, evapotranspiración y radiación solar
    era5_coll = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).filterBounds(geom)

    # Verificar si hay datos disponibles
    ndvi_size = ndvi_coll.size().getInfo()
    if ndvi_size == 0:
        print(f"  ⚠️ No hay datos NDVI disponibles para este rango de fechas")
        return []

    # Obtener fechas disponibles (NDVI cada 16 días)
    fechas_ms = ndvi_coll.aggregate_array('system:time_start').getInfo()
    print(f"  📊 {len(fechas_ms)} imágenes NDVI encontradas")

    datos = []
    errores = 0

    for i, fecha_ms in enumerate(fechas_ms):
        try:
            fecha = pd.to_datetime(fecha_ms, unit='ms')
            fecha_str = fecha.strftime('%Y-%m-%d')

            # Crear ventana de tiempo
            fecha_inicio_ventana = ee.Date(fecha_ms)
            fecha_fin_ventana = fecha_inicio_ventana.advance(1, 'day')

            # NDVI (escalar por 0.0001) - CLAVE: especificar CRS
            ndvi_img = ndvi_coll.filterDate(fecha_inicio_ventana, fecha_fin_ventana).first()
            ndvi_val = ndvi_img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=250,
                crs='EPSG:4326',  # WGS84
                maxPixels=1e9
            ).get('NDVI').getInfo()
            ndvi_val = ndvi_val * 0.0001 if ndvi_val is not None else None

            # LST (escalar por 0.02 y convertir a Celsius)
            lst_img = lst_coll.filterDate(fecha_inicio_ventana.advance(-1, 'day'),
                                          fecha_fin_ventana.advance(1, 'day')).first()
            if lst_img:
                lst_dict = lst_img.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=region,
                    scale=1000,
                    crs='EPSG:4326',
                    maxPixels=1e9
                ).getInfo()
                lst_day = (lst_dict.get('LST_Day_1km') * 0.02 - 273.15) if lst_dict.get('LST_Day_1km') else None
                lst_night = (lst_dict.get('LST_Night_1km') * 0.02 - 273.15) if lst_dict.get('LST_Night_1km') else None
            else:
                lst_day, lst_night = None, None

            # Precipitación (acumulada en ventana de 16 días)
            precip_imgs = precip_coll.filterDate(fecha_inicio_ventana.advance(-8, 'day'),
                                                  fecha_fin_ventana.advance(8, 'day'))
            if precip_imgs.size().getInfo() > 0:
                precip_sum = precip_imgs.sum()
                precip_val = precip_sum.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=region,
                    scale=5000,
                    crs='EPSG:4326',
                    maxPixels=1e9
                ).get('precipitation').getInfo()
            else:
                precip_val = None

            # ERA5-Land (promedio del día)
            era5_day = era5_coll.filterDate(fecha_inicio_ventana.advance(-12, 'hour'),
                                           fecha_fin_ventana.advance(12, 'hour'))
            if era5_day.size().getInfo() > 0:
                era5_mean = era5_day.mean()
                era5_dict = era5_mean.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=region,
                    scale=11132,
                    crs='EPSG:4326',
                    maxPixels=1e9
                ).getInfo()

                soil_moist = era5_dict.get('volumetric_soil_water_layer_1')
                evapotransp = era5_dict.get('evaporation_from_vegetation_transpiration')
                rad_solar = era5_dict.get('surface_solar_radiation_downwards')
            else:
                soil_moist, evapotransp, rad_solar = None, None, None

            datos.append({
                "fecha": fecha_str,
                "NDVI": round(ndvi_val, 4) if ndvi_val else None,
                "LST_Day_C": round(lst_day, 2) if lst_day else None,
                "LST_Night_C": round(lst_night, 2) if lst_night else None,
                "Precipitacion_mm_16dias": round(precip_val, 2) if precip_val else None,
                "Humedad_suelo_m3m3": round(soil_moist, 4) if soil_moist else None,
                "Evapotranspiracion_m": round(evapotransp, 6) if evapotransp else None,
                "Radiacion_solar_Jm2": round(rad_solar, 2) if rad_solar else None
            })

            # Progreso cada 10 imágenes
            if (i + 1) % 10 == 0 or (i + 1) == len(fechas_ms):
                print(f"  Progreso: {i+1}/{len(fechas_ms)} imágenes procesadas")

        except Exception as e:
            errores += 1
            if errores <= 5:  # Mostrar primeros 5 errores
                print(f"  ⚠️ Error en fecha {fecha_str}: {str(e)[:100]}")
            continue

    if errores > 0:
        print(f"  ⚠️ Total de errores: {errores}/{len(fechas_ms)}")

    return datos


In [54]:
# Procesar todas las ubicaciones
resultados_resumen = []

for lugar in ubicaciones_fenomenos:
    print(f"\n{'='*60}")
    print(f"📍 Procesando: {lugar['ubicacion']} ({lugar['pais_region']})")
    print(f"   Coordenadas: {lugar['coordenadas']}")
    print(f"   Período original: {lugar['fecha_inicio']} a {lugar['fecha_fin']}")
    print(f"   Período ajustado: {lugar['fecha_inicio_ajustada']} a {lugar['fecha_fin_ajustada']}")
    print(f"{'='*60}")

    try:
        datos = extraer_datos_ubicacion(lugar)

        if len(datos) > 0:
            # Guardar JSON
            json_file = f"{lugar['ubicacion'].replace(' ','_').replace('/','_')}.json"
            with open(json_file, 'w', encoding='utf-8') as f:
                json.dump({
                    "metadata": {
                        "ubicacion": lugar['ubicacion'],
                        "coordenadas": lugar['coordenadas'],
                        "fenomeno": lugar['fenomeno'],
                        "descripcion": lugar['descripcion'],
                        "periodo_original": f"{lugar['fecha_inicio']} a {lugar['fecha_fin']}",
                        "periodo_ajustado": f"{lugar['fecha_inicio_ajustada']} a {lugar['fecha_fin_ajustada']}"
                    },
                    "datos": datos
                }, f, indent=2, ensure_ascii=False)

            # Guardar CSV
            df = pd.DataFrame(datos)
            csv_file = f"{lugar['ubicacion'].replace(' ','_').replace('/','_')}.csv"
            df.to_csv(csv_file, index=False, encoding='utf-8')

            print(f"\n✅ Guardado exitosamente:")
            print(f"   📄 {json_file} ({len(datos)} registros)")
            print(f"   📊 {csv_file}")

            # Mostrar estadísticas básicas
            print(f"\n📈 Estadísticas básicas:")
            if df['NDVI'].notna().any():
                print(f"   NDVI: min={df['NDVI'].min():.3f}, max={df['NDVI'].max():.3f}, media={df['NDVI'].mean():.3f}")
            if df['LST_Day_C'].notna().any():
                print(f"   Temp día: min={df['LST_Day_C'].min():.1f}°C, max={df['LST_Day_C'].max():.1f}°C, media={df['LST_Day_C'].mean():.1f}°C")
            if df['Precipitacion_mm_16dias'].notna().any():
                print(f"   Precip (16d): min={df['Precipitacion_mm_16dias'].min():.1f}mm, max={df['Precipitacion_mm_16dias'].max():.1f}mm")

            resultados_resumen.append({
                "ubicacion": lugar['ubicacion'],
                "registros": len(datos),
                "archivo": csv_file,
                "status": "✅ OK"
            })
        else:
            print(f"\n❌ No se pudieron extraer datos para esta ubicación")
            resultados_resumen.append({
                "ubicacion": lugar['ubicacion'],
                "registros": 0,
                "archivo": "-",
                "status": "❌ Sin datos"
            })

    except Exception as e:
        print(f"\n❌ Error fatal procesando {lugar['ubicacion']}: {str(e)}")
        resultados_resumen.append({
            "ubicacion": lugar['ubicacion'],
            "registros": 0,
            "archivo": "-",
            "status": f"❌ Error: {str(e)[:50]}"
        })

print(f"\n{'='*60}")
print("🎉 PROCESO COMPLETADO")
print(f"{'='*60}")
print("\n📋 Resumen de resultados:")
for resultado in resultados_resumen:
    print(f"  {resultado['status']} {resultado['ubicacion']}: {resultado['registros']} registros")
print(f"\n{'='*60}")


📍 Procesando: Valle Central / Fresno (California, EE. UU.)
   Coordenadas: [36.7378, -119.7871]
   Período original: 2014-01-01 a 2017-12-31
   Período ajustado: 2014-01-01 a 2017-12-31
  📊 92 imágenes NDVI encontradas
  Progreso: 10/92 imágenes procesadas
  Progreso: 20/92 imágenes procesadas
  Progreso: 30/92 imágenes procesadas
  Progreso: 40/92 imágenes procesadas
  Progreso: 50/92 imágenes procesadas
  Progreso: 60/92 imágenes procesadas
  Progreso: 70/92 imágenes procesadas
  Progreso: 80/92 imágenes procesadas
  Progreso: 90/92 imágenes procesadas
  Progreso: 92/92 imágenes procesadas

✅ Guardado exitosamente:
   📄 Valle_Central___Fresno.json (92 registros)
   📊 Valle_Central___Fresno.csv

📈 Estadísticas básicas:

📍 Procesando: Melbourne / Victoria (Australia (Sur y Este))
   Coordenadas: [-37.8136, 144.9631]
   Período original: 2019-01-01 a 2020-12-31
   Período ajustado: 2018-07-03 a 2021-07-01
  📊 69 imágenes NDVI encontradas
  Progreso: 10/69 imágenes procesadas
  Progreso

# Prueba aparte para cada dato


In [55]:
def diagnosticar_disponibilidad(lugar):
    """Diagnostica qué datos están disponibles para una ubicación"""
    geom = ee.Geometry.Point(lugar["coordenadas"])
    start = lugar["fecha_inicio_ajustada"]
    end = lugar["fecha_fin_ajustada"]

    print(f"\n🔍 Diagnóstico para {lugar['ubicacion']}:")
    print(f"   Coordenadas: {lugar['coordenadas']}")

    # Verificar cada colección
    colecciones = {
        "NDVI (MOD13Q1)": ee.ImageCollection('MODIS/006/MOD13Q1').filterDate(start, end).filterBounds(geom),
        "LST (MOD11A1)": ee.ImageCollection('MODIS/006/MOD11A1').filterDate(start, end).filterBounds(geom),
        "Precipitación (CHIRPS)": ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(start, end).filterBounds(geom),
        "ERA5-Land": ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).filterBounds(geom)
    }

    for nombre, coll in colecciones.items():
        try:
            count = coll.size().getInfo()
            if count > 0:
                first = coll.first()
                sample = first.reduceRegion(
                    reducer=ee.Reducer.first(),
                    geometry=geom.buffer(1000),
                    scale=1000,
                    crs='EPSG:4326'
                ).getInfo()
                print(f"   ✅ {nombre}: {count} imágenes - Ejemplo: {list(sample.keys())}")
            else:
                print(f"   ❌ {nombre}: Sin datos disponibles")
        except Exception as e:
            print(f"   ❌ {nombre}: Error - {str(e)[:80]}")

# Ejecutar diagnóstico para la primera ubicación
diagnosticar_disponibilidad(ubicaciones_fenomenos[0])


🔍 Diagnóstico para Valle Central / Fresno:
   Coordenadas: [36.7378, -119.7871]
   ✅ NDVI (MOD13Q1): 92 imágenes - Ejemplo: ['DayOfYear', 'DetailedQA', 'EVI', 'NDVI', 'RelativeAzimuth', 'SolarZenith', 'SummaryQA', 'ViewZenith', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07']
   ✅ LST (MOD11A1): 1451 imágenes - Ejemplo: ['Clear_day_cov', 'Clear_night_cov', 'Day_view_angle', 'Day_view_time', 'Emis_31', 'Emis_32', 'LST_Day_1km', 'LST_Night_1km', 'Night_view_angle', 'Night_view_time', 'QC_Day', 'QC_Night']
   ✅ Precipitación (CHIRPS): 1460 imágenes - Ejemplo: ['precipitation']
   ✅ ERA5-Land: 35040 imágenes - Ejemplo: ['dewpoint_temperature_2m', 'evaporation_from_bare_soil', 'evaporation_from_bare_soil_hourly', 'evaporation_from_open_water_surfaces_excluding_oceans', 'evaporation_from_open_water_surfaces_excluding_oceans_hourly', 'evaporation_from_the_top_of_canopy', 'evaporation_from_the_top_of_canopy_hourly', 'evaporation_from_vegetation_transpiration', 'evaporation_from

In [ ]:
def extraer_datos_ubicacion_v2(lugar):
    geom = ee.Geometry.Point(lugar["coordenadas"])
    region = geom.buffer(5000)  # 5km buffer para mejor cobertura
    start = lugar["fecha_inicio_ajustada"]
    end = lugar["fecha_fin_ajustada"]

    print(f"  📡 Extrayendo datos...")

    # 1. NDVI - cada 16 días
    print(f"  🌿 Procesando NDVI...")
    ndvi_coll = ee.ImageCollection('MODIS/006/MOD13Q1')\
        .filterDate(start, end)\
        .filterBounds(geom)\
        .select('NDVI')

    ndvi_list = ndvi_coll.toList(1000)
    ndvi_size = ndvi_coll.size().getInfo()

    datos_ndvi = []
    for i in range(min(ndvi_size, 1000)):
        try:
            img = ee.Image(ndvi_list.get(i))
            fecha_ms = img.get('system:time_start').getInfo()
            fecha = pd.to_datetime(fecha_ms, unit='ms').strftime('%Y-%m-%d')

            valor = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=250,
                crs='EPSG:4326',
                maxPixels=1e9
            ).get('NDVI').getInfo()

            if valor is not None:
                datos_ndvi.append({
                    'fecha': fecha,
                    'NDVI': round(valor * 0.0001, 4)
                })
        except:
            continue

    print(f"     ✓ {len(datos_ndvi)} registros NDVI")

    # 2. LST - diario
    print(f"  🌡️  Procesando LST...")
    lst_coll = ee.ImageCollection('MODIS/006/MOD11A1')\
        .filterDate(start, end)\
        .filterBounds(geom)\
        .select(['LST_Day_1km', 'LST_Night_1km'])

    # Muestrear cada 8 días para no sobrecargar
    lst_list = lst_coll.toList(500)
    lst_size = min(lst_coll.size().getInfo(), 500)

    datos_lst = []
    for i in range(0, lst_size, 8):  # Cada 8 días
        try:
            img = ee.Image(lst_list.get(i))
            fecha_ms = img.get('system:time_start').getInfo()
            fecha = pd.to_datetime(fecha_ms, unit='ms').strftime('%Y-%m-%d')

            valores = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=1000,
                crs='EPSG:4326',
                maxPixels=1e9
            ).getInfo()

            lst_day = valores.get('LST_Day_1km')
            lst_night = valores.get('LST_Night_1km')

            if lst_day is not None and lst_night is not None:
                datos_lst.append({
                    'fecha': fecha,
                    'LST_Day_C': round(lst_day * 0.02 - 273.15, 2),
                    'LST_Night_C': round(lst_night * 0.02 - 273.15, 2)
                })
        except:
            continue

    print(f"     ✓ {len(datos_lst)} registros LST")

    # 3. Precipitación - diario
    print(f"  🌧️  Procesando Precipitación...")
    precip_coll = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
        .filterDate(start, end)\
        .filterBounds(geom)\
        .select('precipitation')

    # Muestrear cada 5 días
    precip_list = precip_coll.toList(500)
    precip_size = min(precip_coll.size().getInfo(), 500)

    datos_precip = []
    for i in range(0, precip_size, 5):  # Cada 5 días
        try:
            img = ee.Image(precip_list.get(i))
            fecha_ms = img.get('system:time_start').getInfo()
            fecha = pd.to_datetime(fecha_ms, unit='ms').strftime('%Y-%m-%d')

            valor = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=5000,
                crs='EPSG:4326',
                maxPixels=1e9
            ).get('precipitation').getInfo()

            if valor is not None:
                datos_precip.append({
                    'fecha': fecha,
                    'Precipitacion_mm': round(valor, 2)
                })
        except:
            continue

    print(f"     ✓ {len(datos_precip)} registros Precipitación")

    # 4. ERA5-Land - horario (muestrear semanalmente)
    print(f"  💧 Procesando ERA5-Land...")
    era5_coll = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')\
        .filterDate(start, end)\
        .filterBounds(geom)

    # Agrupar por día y promediar
    dias = pd.date_range(start, end, freq='7D')  # Cada 7 días

    datos_era5 = []
    for dia in dias[:100]:  # Limitar a 100 muestras
        try:
            dia_str = dia.strftime('%Y-%m-%d')
            dia_siguiente = (dia + pd.Timedelta(days=1)).strftime('%Y-%m-%d')

            dia_data = era5_coll.filterDate(dia_str, dia_siguiente).mean()

            valores = dia_data.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=11132,
                crs='EPSG:4326',
                maxPixels=1e9
            ).getInfo()

            if valores:
                datos_era5.append({
                    'fecha': dia_str,
                    'Humedad_suelo_m3m3': round(valores.get('volumetric_soil_water_layer_1', 0), 4) if valores.get('volumetric_soil_water_layer_1') else None,
                    'Evapotranspiracion_m': round(valores.get('evaporation_from_vegetation_transpiration', 0), 6) if valores.get('evaporation_from_vegetation_transpiration') else None,
                    'Radiacion_solar_Jm2': round(valores.get('surface_solar_radiation_downwards', 0), 2) if valores.get('surface_solar_radiation_downwards') else None
                })
        except:
            continue

    print(f"     ✓ {len(datos_era5)} registros ERA5-Land")

    # Combinar todos los datos usando merge en pandas
    df_ndvi = pd.DataFrame(datos_ndvi)
    df_lst = pd.DataFrame(datos_lst)
    df_precip = pd.DataFrame(datos_precip)
    df_era5 = pd.DataFrame(datos_era5)

    # Merge progresivo
    if len(df_ndvi) > 0:
        df_final = df_ndvi
        if len(df_lst) > 0:
            df_final = df_final.merge(df_lst, on='fecha', how='outer')
        if len(df_precip) > 0:
            df_final = df_final.merge(df_precip, on='fecha', how='outer')
        if len(df_era5) > 0:
            df_final = df_final.merge(df_era5, on='fecha', how='outer')

        df_final = df_final.sort_values('fecha').reset_index(drop=True)
        return df_final
    else:
        return pd.DataFrame()

# Procesar ubicaciones
for lugar in ubicaciones_fenomenos:
    print(f"\n{'='*60}")
    print(f"📍 {lugar['ubicacion']} ({lugar['pais_region']})")
    print(f"   {lugar['fecha_inicio_ajustada']} → {lugar['fecha_fin_ajustada']}")
    print(f"{'='*60}")

    try:
        df = extraer_datos_ubicacion_v2(lugar)

        if len(df) > 0:
            # Guardar CSV
            csv_file = f"{lugar['ubicacion'].replace(' ','_').replace('/','_')}.csv"
            df.to_csv(csv_file, index=False, encoding='utf-8')

            print(f"\n✅ Guardado: {csv_file}")
            print(f"   📊 {len(df)} filas totales")
            print(f"\n📈 Completitud de datos:")
            for col in df.columns:
                if col != 'fecha':
                    completitud = (df[col].notna().sum() / len(df)) * 100
                    print(f"   {col}: {completitud:.1f}%")
        else:
            print(f"\n❌ No se extrajeron datos")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()


📍 Valle Central / Fresno (California, EE. UU.)
   2014-01-01 → 2017-12-31
  📡 Extrayendo datos...
  🌿 Procesando NDVI...
     ✓ 0 registros NDVI
  🌡️  Procesando LST...
     ✓ 0 registros LST
  🌧️  Procesando Precipitación...
     ✓ 0 registros Precipitación
  💧 Procesando ERA5-Land...
     ✓ 100 registros ERA5-Land

❌ No se extrajeron datos

📍 Melbourne / Victoria (Australia (Sur y Este))
   2018-07-03 → 2021-07-01
  📡 Extrayendo datos...
  🌿 Procesando NDVI...
     ✓ 0 registros NDVI
  🌡️  Procesando LST...
     ✓ 0 registros LST
  🌧️  Procesando Precipitación...
     ✓ 0 registros Precipitación
  💧 Procesando ERA5-Land...
     ✓ 100 registros ERA5-Land

❌ No se extrajeron datos

📍 Ludhiana (India (Punjab))
   2019-01-01 → 2021-12-31
  📡 Extrayendo datos...
  🌿 Procesando NDVI...
     ✓ 0 registros NDVI
  🌡️  Procesando LST...
     ✓ 0 registros LST
  🌧️  Procesando Precipitación...
     ✓ 0 registros Precipitación
  💧 Procesando ERA5-Land...
     ✓ 100 registros ERA5-Land

❌ No se 